In [ ]:

#pip install numpy
#pip install pandas
#pathul csv-urilor din clona voastra 
#connections = pd.read_csv("D:/visual/challenge/eval-platform/src/main/resources/liquibase/data/connections.csv", delimiter =';')
#customers = pd.read_csv("D:/visual/challenge/eval-platform/src/main/resources/liquibase/data/customers.csv", delimiter=';')
#refineries = pd.read_csv("D:/visual/challenge/eval-platform/src/main/resources/liquibase/data/refineries.csv", delimiter=';')
#demands = pd.read_csv ("D:/visual/challenge/eval-platform/src/main/resources/liquibase/data/demands.csv", delimiter = ';')
#tanks = pd.read_csv("D:/visual/challenge/eval-platform/src/main/resources/liquibase/data/tanks.csv", delimiter = ';')
import numpy as np
from collections import defaultdict

# Create an optimized connections dictionary
valid_connections = {
    (conn['from_id'], conn['to_id']): {
        'distance': conn['distance'],
        'lead_time_days': conn['lead_time_days'],
        'max_capacity': conn['max_capacity'],
        'connection_type': conn['connection_type']
    }
    for _, conn in connections.iterrows()
}

# Pre-sort demands by post_day once
sorted_demands = demands.sort_values('post_day').to_dict('records')

# Create time windows index for faster conflict checking
def create_time_windows_index(jobs):
    # For each connection, store the time windows using it
    connection_windows = defaultdict(list)
    for job in jobs:
        key = (job['from_id'], job['to_id'])
        connection_windows[key].append((job['start'], job['end'], job['quantity']))
    return connection_windows

def is_feasible_schedule(job, connection_windows):
    key = (job['from_id'], job['to_id'])
    if job['quantity'] > valid_connections[key]['max_capacity']:
        return False
    
    # Check for time conflicts
    for start, end, _ in connection_windows[key]:
        if not (job['end'] < start or job['start'] > end):
            return False
    return True

def schedule_jobs_greedy():
    scheduled_jobs = []
    connection_windows = defaultdict(list)
    
    for demand in sorted_demands:
        best_connection = None
        min_distance = float('inf')
        
        # Find the best feasible connection for this demand
        for conn_key, conn_details in valid_connections.items():
            job = {
                'id': demand['id'],
                'from_id': conn_key[0],
                'to_id': conn_key[1],
                'start': demand['start_delivery_day'],
                'end': demand['end_delivery_day'],
                'quantity': demand['quantity'],
                'post_day': demand['post_day'],
                **conn_details
            }
            
            if is_feasible_schedule(job, connection_windows):
                if conn_details['distance'] < min_distance:
                    min_distance = conn_details['distance']
                    best_connection = job
        
        # If found a feasible connection, schedule it
        if best_connection:
            scheduled_jobs.append(best_connection)
            key = (best_connection['from_id'], best_connection['to_id'])
            connection_windows[key].append((
                best_connection['start'],
                best_connection['end'],
                best_connection['quantity']
            ))
    
    return scheduled_jobs

# Execute the optimized scheduling
scheduled_jobs = schedule_jobs_greedy()

# Efficient analysis function
def analyze_schedule(scheduled_jobs):
    total_quantity = 0
    connection_stats = defaultdict(lambda: {'count': 0, 'quantity': 0, 'distance': 0})
    
    for job in scheduled_jobs:
        total_quantity += job['quantity']
        conn_type = job['connection_type']
        connection_stats[conn_type]['count'] += 1
        connection_stats[conn_type]['quantity'] += job['quantity']
        connection_stats[conn_type]['distance'] += job['distance']
    
    return total_quantity, dict(connection_stats)

import json
from collections import defaultdict

# Create connection map
connection_map = {
    (row['from_id'], row['to_id']): row['id']
    for _, row in connections.iterrows()
}

def format_scheduled_movements(scheduled_jobs):
    # Find the absolute min and max days from demands dataset
    min_day = int(demands['start_delivery_day'].min())
    max_day = int(demands['end_delivery_day'].max())
    
    # Initialize all days with empty movements
    daily_movements = {day: [] for day in range(min_day, max_day + 1)}
    
    # Process each scheduled job
    for job in scheduled_jobs:
        # Calculate daily amount
        days_count = job['end'] - job['start'] + 1
        daily_amount = job['quantity'] / days_count
        
        # Add movement to each day in the window
        for day in range(job['start'], job['end'] + 1):
            conn_id = connection_map[(job['from_id'], job['to_id'])]
            
            movement = {
                "connectionId": conn_id,
                "amount": daily_amount
            }
            daily_movements[day].append(movement)
    
    # Convert to list format, including ALL days
    schedule = [
        {
            "day": day,
            "movements": movements  # This will be an empty list for days with no movements
        }
        for day, movements in sorted(daily_movements.items())
    ]
    
    return schedule

# Execute scheduling
scheduled_jobs = schedule_jobs_greedy()

# Format and get the schedule
schedule = format_scheduled_movements(scheduled_jobs)

# Print in JSON format
print(json.dumps(schedule, indent=2))